In [1]:
import pandas as pd
import jenkspy
import numpy as np
from owlready2 import *
import ast
import rdflib
import re

owlready2.JAVA_EXE ="/usr/lib/jvm/java-17-openjdk-amd64/bin/java"

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
df = pd.read_csv("test_dataset.csv")
df.fillna(0,inplace=True)

In [15]:
print(df.shape)

(35, 41)


In [3]:
def convert_and_round(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return value  

In [4]:
for col in df.columns:
    df[col] = df[col].apply(convert_and_round)

In [5]:
df.shape[1]

41

In [6]:
onto = get_ontology("file:///home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/onto/MtL_Enriched.owl").load()
dcat = get_namespace("http://www.w3.org/ns/dcat/")
dqv = get_namespace("http://www.w3.org/ns/dqv/")
mtl = get_ontology("https://purl.archive.org/domain/mtl#")
#onto.base_iri = "https://purl.archive.org/domain/mtl#"
rdf=get_namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
dmop = get_ontology("http://www.e-lico.eu/ontologies/dmop/DMOP/DMOP.owl")
dmop.base_iri = "http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#"
dmop1 = get_ontology("http://www.e-LICO.eu/ontologies/dmo/DMOP/DMOP.owl")
dmop1.base_iri = "http://www.e-LICO.eu/ontologies/dmo/DMOP/DMOP.owl#"

#sync_reasoner_pellet([onto])

In [7]:
pattern = re.compile(r'[^a-zA-Z0-9]') #removing non-alpha numeric characters from file name

fileObjects = {}
df.shape

(35, 41)

In [8]:
with onto:
    for i in range(len(df)):
  #      i=34
        fileName = df['File'][i]
        fileName = pattern.sub('', fileName)

        mtla = mtl.LearningFromTaskProperties(fileName, namespace=onto)
        mtlt = mtl.FeatureSelectionRecommendationTask(fileName, namespace=onto)

        fileObject = dmop.LabeledDataSet(fileName, namespace=onto)
        fileObjects[fileName] = mtlt
        fsr_output = mtl.FeatureSelectionRecommendation(fileName, namespace=onto)
        mtla.addresses.append(mtlt)

        mtlt.specifiesInputClass = [fileObject]
        mtlt.specifiesOutputClass = [fsr_output]

        nInstance = dmop1.NumberOfInstances(fileName, namespace=onto)
        fileObject.hasQuality.append(nInstance)
        q3 = mtl.Mean(fileName+"mean1")
        nInstance.qlocation.append(q3) 
        q3.hasDatavalue = [int(df['nr_inst'][i])]

        rRange = mtl.Range(fileName)
        fileObject.hasQuality.append(rRange)
        q2 = mtl.Mean(fileName+"mean2")
        rRange.qlocation.append(q2)
        q2.hasDatavalue  = [float(df['range.mean'][i])]

        attrConc = mtl.AttributeConcentration(fileName)
        fileObject.hasQuality.append(attrConc)
        q4 = mtl.Mean(fileName+"mean3")
        attrConc.qlocation.append(q4)
        q4.hasDatavalue  = [float(df['attr_conc.mean'][i])]

        covR = mtl.Covariance(fileName)
        fileObject.hasQuality.append(covR)
        q5 = mtl.Mean(fileName+"mean4")
        covR.qlocation.append(q5)
        q5.hasDatavalue = [float(df['cov.mean'][i])]

        cEntr = mtl.ClassEntropy(fileName)
        fileObject.hasQuality.append(cEntr)
        q6 = mtl.Mean(fileName+"mean5")
        cEntr.qlocation.append(q6)
        q6.hasDatavalue  = [ float(df['cEntropy'][i])]

        outDet = mtl.OutlierDetection(fileName)
        fileObject.hasQuality.append(outDet)
        q7 = mtl.Mean(fileName+"mean6")
        outDet.qlocation.append(q7)
        q7.hasDatavalue = [float(df['OutlierPerc'][i])]

        nrOut = mtl.NumberOfOutliers(fileName)
        fileObject.hasQuality.append(nrOut)
        q8 = mtl.Mean(fileName+"mean7")
        nrOut.qlocation.append(q8)
        q8.hasDatavalue = [float(df['nr_outliers'][i])]

        instFeat = dmop1.ProportionOfInstancesPerFeature(fileName, namespace=onto)
        fileObject.hasQuality.append(instFeat)
        q12 = mtl.Mean(fileName+"mean8")
        instFeat.qlocation.append(q12)
        q12.hasDatavalue  = [float(df['inst_to_attr'][i])]

        attrEntr = mtl.AttributesEntropy(fileName)
        fileObject.hasQuality.append(attrEntr)
        q9 = mtl.Mean(fileName+"mean9")
        attrEntr.qlocation.append(q9)
        q9.hasDatavalue  = [float(df['attr_ent.mean'][i])]

        corr = mtl.Correlation("corr", namespace=onto)
        fileObject.hasQuality.append(corr)
        q10 = mtl.Mean(fileName+"mean10")
        corr.qlocation.append(q10)
        q10.hasDatavalue  = [float(df['cor.mean'][i])]

        medAttr = mtl.MedianOfAttributes(fileName)
        fileObject.hasQuality.append(medAttr)
        q11 = mtl.Mean(fileName+"mean11")
        medAttr.qlocation.append(q11)
        q11.hasDatavalue  = [float(df['median.mean'][i])]

        classImb = mtl.ClassImbalance(fileName)
        fileObject.hasQuality.append(classImb)
        q13 = mtl.Mean(fileName+"mean12")
        classImb.qlocation.append(q13)
        q13.hasDatavalue  = [float(df['ClassImbRatio'][i])]

        ena = mtl.EqNumberOfAttributes(fileName)
        fileObject.hasQuality.append(ena)
        q14 = mtl.Mean("mean13")
        ena.qlocation.append(q14)
        q14.hasDatavalue  = [float(df['ena'][i])]

        labelNoise = mtl.LabelNoise(fileName)
        fileObject.hasQuality.append(labelNoise)
        q15 = mtl.Mean(fileName+"mean14")
        labelNoise.qlocation.append(q15)
        q15.hasDatavalue  = [float(df['LabelIssuesPerc'][i])]

In [9]:
try:    
    sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True)
    print("✅ Ontology is consistent.")
except OwlReadyInconsistentOntologyError:
    print("❌ Ontology inconsistent! Checking problem individuals...")
    for ind in onto.individuals():
        if ind.is_instance_of:
            print(f"Individual: {ind}, Classes: {ind.is_instance_of}")
except Exception as e:
    print("Reasoner failed:", e)

* Owlready2 * Running Pellet...
    /usr/lib/jvm/java-17-openjdk-amd64/bin/java -Xmx2000M -cp /home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/antlr-3.2.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/httpcore-4.2.2.jar:/home/d19125691/.local/

* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean3
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean7
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean9
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean4
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean1
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean5
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.breastcancerwdbcdatamean12
* Owlready * Adding relation MtL_Enriched.breastcancerwdbcdata immediate-relation MtL_Enriched.mean13
* Owlready * Adding relation MtL_Enriched.breastca

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [10]:
# Check classification again just to be safe
for fileName, fileObject in fileObjects.items():
    if onto.MIRelevantData in fileObject.is_a:
        print(f"{fileObject.name} satisfies MIRelevantData constraint")
        output = onto.MITask.specifiesOutputClass
        print("Recommended feature selection technique is", output[0].recommends)
    if onto.FCBFRelevantData in fileObject.is_a:
        print(f"{fileObject.name} satisfies FCBFRelevantData constraint")
        output = onto.FCBFTask.specifiesOutputClass
        print("Recommended feature selection technique is", output[0].recommends)
    if onto.GRRelevantData in fileObject.is_a:
        print(f"{fileObject.name} satisfies GRRelevantData constraint")
        output = onto.GRTask.specifiesOutputClass
        print("Recommended feature selection technique is", output[0].recommends)
    if onto.ReliefRelevantData in fileObject.is_a:
        print(f"{fileObject.name} satisfies ReliefRelevantData constraint")
        output = onto.ReliefTask.specifiesOutputClass
        print("Recommended feature selection technique is", output[0].recommends)
    if onto.ChisquareRelevantData in fileObject.is_a:
        print(f"{fileObject.name} satisfies ChisquareRelevantData constraint")
        output = onto.ChisquareTask.specifiesOutputClass
        print("Recommended feature selection technique is", output[0].recommends)

yeast2vs4csv satisfies ReliefRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.Relief]
yeast2vs4csv satisfies ChisquareRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.ChiSquare]
ckddatasetv2csv satisfies GRRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.GainRatio]
ckddatasetv2csv satisfies ReliefRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.Relief]
ckddatasetv2csv satisfies ChisquareRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.ChiSquare]
HTRU2csv satisfies GRRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.GainRatio]
HTRU2csv satisfies ReliefRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.Relief]
HTRU2csv satisfies ChisquareRelevantData constraint
Recommended feature selection technique is [MtL_Enriched.ChiSquare]
ecolidata satisfies GRRelevantData constrai

In [11]:
inferred_classes = mtla.is_a
print(mtla.is_a)

recommendation_classes = [mtl.MIRelevantData, mtl.ChisquareRelevantData, mtl.GRRelevantData, mtl.FCBFRelevantData, 
mtl.ReliefRelevantData]

for cls in inferred_classes:
    if cls in recommendation_classes:
        print(f"Recommended feature selection technique is {cls.recommends}") 
        break  
else:
    print("No specific feature selection technique was recommended.")

[MtL_Enriched.LearningFromTaskProperties, MtL_Enriched.FeatureSelectionRecommendationTask, MtL_Enriched.FeatureSelectionRecommendation, MtL_Enriched.MIMean2, MtL_Enriched.ReliefMean4, MtL_Enriched.GRMean2, MtL_Enriched.MIMean3, MtL_Enriched.ReliefMean7, MtL_Enriched.ReliefMean5, MtL_Enriched.ChisquareMean2, MtL_Enriched.ReliefRelevantData, MtL_Enriched.GRMean3, MtL_Enriched.FCBFMean3, MtL_Enriched.FCBFMean5, MtL_Enriched.ChisquareMean4, MtL_Enriched.ReliefMean6, MtL_Enriched.FCBFMean4, MtL_Enriched.GRMean5, MtL_Enriched.ChisquareRelevantData, MtL_Enriched.ReliefMean1, MtL_Enriched.GRMean4, MtL_Enriched.GRRelevantData, MtL_Enriched.FCBFMean6]
Recommended feature selection technique is []


# Upload triples to Fuseki

In [12]:
'''import requests

fuseki_url = "http://localhost:3030/KB/data" #URL of the Fuseki dataset

rdf_file = "Populated.rdf"

with open(rdf_file, 'rb') as file:
    rdf_data = file.read()

response = requests.post(fuseki_url, data=rdf_data, headers={'Content-Type': 'application/rdf+xml'})

if response.status_code == 200 or response.status_code == 204:
    print("Data uploaded successfully")
else:
    print(f"Failed to upload data. Status code: {response.status_code}")
    print(f"Response: {response.text}")'''

'import requests\n\nfuseki_url = "http://localhost:3030/KB/data" #URL of the Fuseki dataset\n\nrdf_file = "Populated.rdf"\n\nwith open(rdf_file, \'rb\') as file:\n    rdf_data = file.read()\n\nresponse = requests.post(fuseki_url, data=rdf_data, headers={\'Content-Type\': \'application/rdf+xml\'})\n\nif response.status_code == 200 or response.status_code == 204:\n    print("Data uploaded successfully")\nelse:\n    print(f"Failed to upload data. Status code: {response.status_code}")\n    print(f"Response: {response.text}")'

# Query to extract limits of correlation metric - simple example

In [13]:
'''from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "http://localhost:3030/KB/sparql" 

sparql = SPARQLWrapper(endpoint_url)
query = """
    PREFIX dmop: <http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#>
    PREFIX dmop1: <http://www.e-LICO.eu/ontologies/dmo/DMOP/DMOP.owl#>
    PREFIX mtl: <https://purl.archive.org/domain/mtl#>  
    SELECT ?upperValue ?lowerValue ?mean ?stdDev
    WHERE {  
        ?metal a mtl:MetaLearningAlgorithm . 
        ?featureSelectionTask a dmop:FeatureSelectionTask .
        ?metal mtl:hasMetaObjective ?featureSelectionTask .
        ?cor a mtl:CorrelationParameter .
        ?metal dmop:hasParameter ?cor .
        ?cor mtl:hasLowerValue ?lowerValue .
        ?cor mtl:hasUpperValue ?upperValue .
        ?m a mtl:Mean .
        ?std a mtl:StdDev .
        ?cor mtl:hasQuality ?m .
        ?cor mtl:hasQuality ?std .
        ?m dmop:hasValue ?mean .
  ?std dmop:hasValue ?stdDev . 
} """

sparql.setQuery(query)

sparql.setReturnFormat(JSON)

results = sparql.query().convert()

if "results" in results and "bindings" in results["results"]:
    for result in results["results"]["bindings"]:
        print(result)
        upperValue = result["upperValue"]["value"]
        lowerValue = result["lowerValue"]["value"]
        print(f"Feature Selection Algorithm: {upperValue}, Feature Count: {lowerValue}")
else:
    print("Unexpected response format. Please check the SPARQL query and endpoint.")'''

'from SPARQLWrapper import SPARQLWrapper, JSON\n\nendpoint_url = "http://localhost:3030/KB/sparql" \n\nsparql = SPARQLWrapper(endpoint_url)\nquery = """\n    PREFIX dmop: <http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#>\n    PREFIX dmop1: <http://www.e-LICO.eu/ontologies/dmo/DMOP/DMOP.owl#>\n    PREFIX mtl: <https://purl.archive.org/domain/mtl#>  \n    SELECT ?upperValue ?lowerValue ?mean ?stdDev\n    WHERE {  \n        ?metal a mtl:MetaLearningAlgorithm . \n        ?featureSelectionTask a dmop:FeatureSelectionTask .\n        ?metal mtl:hasMetaObjective ?featureSelectionTask .\n        ?cor a mtl:CorrelationParameter .\n        ?metal dmop:hasParameter ?cor .\n        ?cor mtl:hasLowerValue ?lowerValue .\n        ?cor mtl:hasUpperValue ?upperValue .\n        ?m a mtl:Mean .\n        ?std a mtl:StdDev .\n        ?cor mtl:hasQuality ?m .\n        ?cor mtl:hasQuality ?std .\n        ?m dmop:hasValue ?mean .\n  ?std dmop:hasValue ?stdDev . \n} """\n\nsparql.setQuery(query)\n\nsparql.se

# Query to extract datasets and associated feature selection techniques

In [14]:
'''from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "http://localhost:3030/KB/sparql" 

sparql = SPARQLWrapper(endpoint_url)
query = """
    PREFIX dmop: <http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#>
    PREFIX dmop1: <http://www.e-LICO.eu/ontologies/dmo/DMOP/DMOP.owl#>
    PREFIX mtl: <https://purl.archive.org/domain/mtl#>  
    SELECT ?featureAlgo ?dataset
    WHERE {  
        ?metal a mtl:MetaLearningAlgorithm . 
        ?featureSelectionTask a dmop:FeatureSelectionTask .
        ?metal mtl:hasMetaObjective ?featureSelectionTask .
  		?dataset a dmop:DataSetClass .   
		?featureSelectionTask dmop:specifiesInputClass ?dataset .
  		?featureAlgoOutput a dmop:StructuredPredictionModelClass .
  		?featureSelectionTask dmop:specifiesOutputClass ?featureAlgoOutput .
  		?featureAlgoOutput dmop:hasValue ?featureAlgo .
} """

sparql.setQuery(query)

sparql.setReturnFormat(JSON)

results = sparql.query().convert()

if "results" in results and "bindings" in results["results"]:
    for result in results["results"]["bindings"]:
        featureAlgo = result["featureAlgo"]["value"]
        dataset = result["dataset"]["value"]
        print(f"Feature Selection Algorithm: {featureAlgo}, Dataset: {dataset}")
else:
    print("Unexpected response format. Please check the SPARQL query and endpoint.")'''

'from SPARQLWrapper import SPARQLWrapper, JSON\n\nendpoint_url = "http://localhost:3030/KB/sparql" \n\nsparql = SPARQLWrapper(endpoint_url)\nquery = """\n    PREFIX dmop: <http://www.e-lico.eu/ontologies/dmo/DMOP/DMOP.owl#>\n    PREFIX dmop1: <http://www.e-LICO.eu/ontologies/dmo/DMOP/DMOP.owl#>\n    PREFIX mtl: <https://purl.archive.org/domain/mtl#>  \n    SELECT ?featureAlgo ?dataset\n    WHERE {  \n        ?metal a mtl:MetaLearningAlgorithm . \n        ?featureSelectionTask a dmop:FeatureSelectionTask .\n        ?metal mtl:hasMetaObjective ?featureSelectionTask .\n  \t\t?dataset a dmop:DataSetClass .   \n\t\t?featureSelectionTask dmop:specifiesInputClass ?dataset .\n  \t\t?featureAlgoOutput a dmop:StructuredPredictionModelClass .\n  \t\t?featureSelectionTask dmop:specifiesOutputClass ?featureAlgoOutput .\n  \t\t?featureAlgoOutput dmop:hasValue ?featureAlgo .\n} """\n\nsparql.setQuery(query)\n\nsparql.setReturnFormat(JSON)\n\nresults = sparql.query().convert()\n\nif "results" in resul